In [21]:
import json
import logging
import requests
import msal
import pandas as pd
config = json.load(open('.env'))

app = msal.ConfidentialClientApplication(
    config["client_id"], authority=config["authority"],
    client_credential=config["secret"],
    )
result = None
result = app.acquire_token_silent(config["scope"], account=None)

if not result:
    logging.info("No suitable token exists in cache. Let's get a new one from AAD.")
    result = app.acquire_token_for_client(scopes=config["scope"])
print(result)

{'token_type': 'Bearer', 'expires_in': 3599, 'ext_expires_in': 3599, 'access_token': 'eyJ0eXAiOiJKV1QiLCJub25jZSI6IlJJZi1NVi1XMGVrWktsckZsUnRZcE1xeTBUdzhHWHRYMW9PeVh2Y0hiNGsiLCJhbGciOiJSUzI1NiIsIng1dCI6ImpTMVhvMU9XRGpfNTJ2YndHTmd2UU8yVnpNYyIsImtpZCI6ImpTMVhvMU9XRGpfNTJ2YndHTmd2UU8yVnpNYyJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC9jN2YxZTNjZS1iYTY2LTQwYTctOTFiZC05NTk0YjM2MjIzZmMvIiwiaWF0IjoxNjUwNDc0NTI2LCJuYmYiOjE2NTA0NzQ1MjYsImV4cCI6MTY1MDQ3ODQyNiwiYWlvIjoiRTJaZ1lHQ1ZMcGdTWlJGdXE1WHI1SzZocytjZEFBPT0iLCJhcHBfZGlzcGxheW5hbWUiOiJHcmFwaCBQeXRob24iLCJhcHBpZCI6ImIzNWFlMTc3LWFlMmEtNGRhOS05MDU2LTdiZWMxNjk1Mzk5MiIsImFwcGlkYWNyIjoiMSIsImlkcCI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0L2M3ZjFlM2NlLWJhNjYtNDBhNy05MWJkLTk1OTRiMzYyMjNmYy8iLCJpZHR5cCI6ImFwcCIsIm9pZCI6IjE1YTkyYWEwLWIwZjYtNDcyYy05N2ViLWM4ZDVmMTU3NTdmOSIsInJoIjoiMC5BWG9BenVQeHgyYTZwMENSdlpXVXMySWpfQU1BQUFBQUFBQUF3QUFBQUFBQUFBQjZBQUEuIiwicm9sZXMiOlsiRGlyZWN0b3J5LlJlYWQuQWxsIiwiVXNlci5SZWFkLkFsbCIsI

In [27]:
url = "https://graph.microsoft.com/beta/users?filter=signInActivity/lastSignInDateTime le 2021-01-01T00:00:00Z"
res_inactiveacct = requests.get(
        url,
        headers={'Authorization': 'Bearer ' + result['access_token']}, ).json()
df_inactiveacct = pd.DataFrame.from_dict(res_inactiveacct['value'])


In [33]:
df_inactiveacct[['id', 'givenName', 'displayName', 'userPrincipalName', 'signInSessionsValidFromDateTime']]

,id,givenName,displayName,userPrincipalName,signInSessionsValidFromDateTime
0,6ed2cd3d-7965-4dc2-b34c-8c59ebb89b65,Quin,Quin,quin@dltapps.co.uk,2020-04-26T07:06:40Z
1,c98da27a-cd17-4d5b-911a-b81c3a268b85,Mona,Mona Reyes,mona.reyes@dltapps.co.uk,2020-08-21T15:48:04Z


In [34]:
df_inactiveacct.columns


Index(['id', 'deletedDateTime', 'accountEnabled', 'ageGroup', 'businessPhones',
       'city', 'createdDateTime', 'creationType', 'companyName',
       'consentProvidedForMinor', 'country', 'department', 'displayName',
       'employeeId', 'employeeHireDate', 'employeeType', 'faxNumber',
       'givenName', 'imAddresses', 'infoCatalogs', 'isManagementRestricted',
       'isResourceAccount', 'jobTitle', 'legalAgeGroupClassification', 'mail',
       'mailNickname', 'mobilePhone', 'onPremisesDistinguishedName',
       'officeLocation', 'onPremisesDomainName', 'onPremisesImmutableId',
       'onPremisesLastSyncDateTime', 'onPremisesSecurityIdentifier',
       'onPremisesSamAccountName', 'onPremisesSyncEnabled',
       'onPremisesUserPrincipalName', 'otherMails', 'passwordPolicies',
       'postalCode', 'preferredDataLocation', 'preferredLanguage',
       'proxyAddresses', 'refreshTokensValidFromDateTime', 'showInAddressList',
       'signInSessionsValidFromDateTime', 'state', 'streetAddres